In [11]:
# Imports
from flax import linen as nn

import jax
import optax
import flax
import jax.numpy as jnp
from jax import jit
from jax import lax
from jax_resnet import pretrained_resnet, slice_variables, Sequential
from flax.jax_utils import replicate, unreplicate
from flax.training import train_state, checkpoints
from flax import linen as nn
from flax.core import FrozenDict,frozen_dict
from flax.training.common_utils import shard

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-11-30 22:39:05.770981: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-30 22:39:05.872565: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-30 22:39:08.891330: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-30 22:39:08.891391: W tensorflow/stream_executor/platform/default/dso_loade

In [ ]:
#Get logits given state
def get_logits(state):
    logits = []
    iter_n = len(test_dataset)
    with tqdm(total=iter_n, desc=f"{iter_n} iterations", leave=False) as progress_bar:
        for _batch in test_dataset:
            batch = _batch[0]
            labels = _batch[1]

            batch = jnp.array(batch, dtype=jnp.float32)
            labels = jnp.array(labels, dtype=jnp.float32)

            batch, labels = shard(batch), shard(labels)
            metric = parallel_val_step(state, batch, labels)[0]
            valid_accuracy.append(metric)
            progress_bar.update(1)
            del(batch)
            del(labels)    

# Get validation accuracy given state
parallel_val_step = jax.pmap(val_step, axis_name='batch', donate_argnums=(0,))
def get_valid_acc(state):
    valid_accuracy = []
    iter_n = len(test_dataset)
    with tqdm(total=iter_n, desc=f"{iter_n} iterations", leave=False) as progress_bar:
        for _batch in test_dataset:
            batch = _batch[0]
            labels = _batch[1]

            batch = jnp.array(batch, dtype=jnp.float32)
            labels = jnp.array(labels, dtype=jnp.float32)

            batch, labels = shard(batch), shard(labels)
            metric = parallel_val_step(state, batch, labels)[0]
            valid_accuracy.append(metric)
            progress_bar.update(1)
            del(batch)
            del(labels)

    avg_valid_acc = sum(valid_accuracy)/len(valid_accuracy)
    avg_valid_acc = np.array(avg_valid_acc)[0]
    valid_acc.append(avg_valid_acc)
    print(f"[{epoch_i+1}/{Config['N_EPOCHS']}] Valid Accuracy: {avg_valid_acc:.03}")
    return avg_valid_acc 

In [ ]:
# Load in the fine-tuned models
def create_train_state(model, variables, optimizer):
  state = TrainState.create(
      apply_fn = model.apply,
      params = variables['params'],
      tx = optimizer,
      batch_stats = variables['batch_stats'],
      loss_fn = loss_fn,
      eval_fn = eval_fn
  )
  
  return state

states = []
for i in range(k):
    model, variables = get_model_and_variables('resnet18', 0)
    state_dict = checkpoints.restore_checkpoint(f'model_{i}', target=None)
    state = create_train_state(model, state_dict, None)
    states.append(state)

In [4]:
# Get all the state accuracies 
def state_accs(states):
    accs = []
    ### BEGIN CODE HERE ###
    for state in states:
        accs.append(get_valid_acc(state))  
    ### END CODE HERE ##
    return accs

In [5]:
# Graph all the model accuracies (label the best one)
valid_accs = state_accs(states)

names = [f'Model {i+1}' for i in range(len(states))]
plt.bar(names, valid_accs)
plt.title("Validation Accuracy By Model")
plt.ylim([0.95,1])
plt.xticks(rotation=30, ha='right')

NameError: name 'states' is not defined

## Ensemble Learning: Average the logits of the models

In [ ]:
# get ensemble learning accruacy 
def ensemble_acc(states):
    acc = None
    for state in states: 
    ### BEGIN CODE HERE ###
        
    ### END CODE HERE ###
    return acc

In [ ]:
# Graph the validation accuracy of ensemble model vs individual models

names = [f'Model {i+1}' for i in range(len(states) + 1)]
plt.bar(names, valid_accs)
plt.title("Validation Accuracy ")
plt.ylim([0.8,1])
plt.xticks(rotation=30, ha='right')

Question: Why do you think the ensemble output has better accuracy than the single models?

Answer: 

## Uniform Soup: Uniformly

In [ ]:
# Return model with weights averaged according to alphas
def get_model(states, alphas):
    # model, _ = clip.load('ViT-B/32')
    # feature_dim = state_dicts[0]['classification_head.weight'].shape[1]
    # num_classes = state_dicts[0]['classification_head.weight'].shape[0]
    # normalize = True
    # model = ModelWrapper(model, feature_dim, num_classes, normalize)
    states.param

    sd = {k : states.params[k] * alphal[0] for k in state_dicts[0].keys()}
    for i in range(1, len(state_dicts)):
        for k in state_dicts[i].keys():
            sd[k] = sd[k] + state_dicts[i][k].clone() * alphal[i]

    model.load_state_dict(sd)
    model = model.to(device)
    return model

In [ ]:
def uniform_soup:
    

## Greedy Soup Implementation

Question: Why does greedy soup perform better than uniform? Do you think this is always the case?

Answer: 

Question: What are the benefits of souping vs traditional ensemble methods in terms of space and time complexity?

Answer: 

## Learned Soup Implementation

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a527420b-c11f-44a4-82a9-68959d523a68' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>